In [19]:
import collections
import math
import os
import shutil
import pandas as pd
import torch
import torchvision
from torch import nn
from d2l import torch as d2l
import time

import os
import shutil
import collections
import math
import time
import torchvision
import torch
import torch.nn as nn
import torch.utils.data

import datetime
import torch.optim as optim


Download data from Kaggle and put them into data folder

* `../data/cifar-10/train/[1-50000].png`
* `../data/cifar-10/test/[1-300000].png`
* `../data/cifar-10/trainLabels.csv`
* `../data/cifar-10/sampleSubmission.csv`


In [34]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim
import time
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader, random_split
import torch.optim as optim


from d2l import torch as d2l  


if torch.cuda.is_available():
    devices = [torch.device('cuda')]
    print(f"Training on {torch.cuda.get_device_name(0)}")
else:
    devices = [torch.device('cpu')]
    print("Training on CPU")



transform_train = transforms.Compose([
    transforms.Resize(40),
    transforms.RandomResizedCrop(32, scale=(0.64, 1.0), ratio=(1.0, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])
])
batch_size = 128


test_ds = torchvision.datasets.CIFAR10(root='../data', train=False, download=True, transform=transform_test)


full_train_ds = torchvision.datasets.CIFAR10(root='../data', train=True, download=True, transform=transform_train)

# split ds
valid_ratio = 0.1  
num_train = len(full_train_ds)
num_valid = int(num_train * valid_ratio)
train_ds, valid_ds = random_split(full_train_ds, [num_train - num_valid, num_valid])


train_iter = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2)
valid_iter = DataLoader(valid_ds, batch_size=batch_size, shuffle=False, num_workers=2)



train_iter = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2)
test_iter = torch.utils.data.DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=2)
valid_ratio = 0.1

valid_iter = torch.utils.data.DataLoader(valid_ds, batch_size, shuffle=False,
                                         drop_last=True)

test_iter = torch.utils.data.DataLoader(test_ds, batch_size, shuffle=False,
                                        drop_last=False)
def get_net():
    num_classes = 10
    net = d2l.resnet18(num_classes, 3)
    return net

loss = nn.CrossEntropyLoss(reduction="none")


   
def train(net1, net2, train_iter, valid_iter, num_epochs, lr, wd, devices, lr_period, lr_decay):
    trainer1 = optim.SGD(net1.parameters(), lr=lr, momentum=0.9, weight_decay=wd)
    trainer2 = optim.SGD(net2.parameters(), lr=lr, momentum=0.9, weight_decay=wd)
    scheduler1 = torch.optim.lr_scheduler.StepLR(trainer1, lr_period, lr_decay)
    scheduler2 = torch.optim.lr_scheduler.StepLR(trainer2, lr_period, lr_decay)
    total_time = 0
    num_batches, timer = len(train_iter), d2l.Timer()
    train_loss_list, train_acc_list, valid_acc_list = [], [], []
    # net1 = nn.DataParallel(net1, device_ids=devices).to(devices[0])
    # net2 = nn.DataParallel(net2, device_ids=devices).to(devices[0])
    for epoch in range(num_epochs):
        start = time.time()
        net1.train()
        net2.train()
        metric1 = d2l.Accumulator(3)
        metric2 = d2l.Accumulator(3)
        # Train net1
        for i, (features, labels) in enumerate(train_iter):
            features, labels = features.to(devices[0]), labels.to(devices[0])
            trainer1.zero_grad()
            outputs = net1(features)
            loss = nn.CrossEntropyLoss()(outputs, labels)
            loss.backward()
            trainer1.step()
            metric1.add(loss * labels.shape[0], d2l.accuracy(outputs, labels), labels.shape[0])

        # Train net2
        for i, (features, labels) in enumerate(train_iter):
            features, labels = features.to(devices[0]), labels.to(devices[0])
            trainer2.zero_grad()
            outputs = net2(features)
            loss = nn.CrossEntropyLoss()(outputs, labels)
            loss.backward()
            trainer2.step()
            metric2.add(loss * labels.shape[0], d2l.accuracy(outputs, labels), labels.shape[0])

        # Validation accuracy
        valid_acc1 = d2l.evaluate_accuracy_gpu(net1, valid_iter)
        valid_acc2 = d2l.evaluate_accuracy_gpu(net2, valid_iter)

        # Update learning rate
        scheduler1.step()
        scheduler2.step()
        epoch_time = time.time() - start
        total_time += epoch_time
        # Print metrics
        train_loss1 = metric1[0] / metric1[2]
        train_acc1 = metric1[1] / metric1[2]
        train_loss2 = metric2[0] / metric2[2]
        train_acc2 = metric2[1] / metric2[2]
        print(f'Epoch {epoch+1}, Net1 loss {train_loss1:.3f}, accuracy {train_acc1:.3f}, Net2 loss {train_loss2:.3f}, accuracy {train_acc2:.3f}, Valid acc1 {valid_acc1:.3f}, Valid acc2 {valid_acc2:.3f}, time {epoch_time:.0f}')


        train_loss_list.append(train_loss1)
        train_acc_list.append(train_acc1)
        valid_acc_list.append(valid_acc1)
    print(f'Time {total_time:.1f} sec')


    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(range(1, num_epochs + 1), train_loss_list, label='train_loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(range(1, num_epochs + 1), train_acc_list, label='train_acc')
    plt.plot(range(1, num_epochs + 1), valid_acc_list, label='valid_acc')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.legend()


    current_time = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
    save_dir = f'training_plots'
    os.makedirs(save_dir, exist_ok=True)
    plot_filename = f"{save_dir}/{current_time}_train_plot_{noise_type}_{noise_level}.png"
    plt.savefig(plot_filename)
    plt.close()

devices, num_epochs, lr, wd = d2l.try_all_gpus(), 100, 2e-4, 5e-4
lr_period, lr_decay, net = 4, 0.9, get_net()

# devices, num_epochs, lr, wd = d2l.try_all_gpus(), 100, 0.05, 5e-4
# lr_period, lr_decay, net = 50, 0.1, get_net()

# devices, num_epochs, lr, wd = d2l.try_all_gpus(), 50, 2e-4, 5e-4
# lr_period, lr_decay, net = 4, 0.9, get_net()
# # devices, num_epochs, lr, wd = d2l.try_all_gpus(), 100, 0.05, 5e-4
# # lr_period, lr_decay, net = 50, 0.1, get_net()
# train(net, train_iter, valid_iter, num_epochs, lr, wd, devices, lr_period,
#       lr_decay)


Training on NVIDIA GeForce GTX 1660 Ti
Files already downloaded and verified
Files already downloaded and verified


In [21]:
import torch
import numpy as np
import random

def add_noise_to_labels(labels, noise_ratio, noise_type='symmetric', num_classes=10):
    if noise_type not in ['symmetric', 'asymmetric']:
        raise ValueError("noise_type should be 'symmetric' or 'asymmetric'")

    noisy_labels = labels.clone()
    n = len(labels)

    # 对称噪声
    if noise_type == 'symmetric':
        for i in range(n):
            if random.random() < noise_ratio:
                noisy_labels[i] = random.choice([l for l in range(num_classes) if l != labels[i]])

    # 非对称噪声
    elif noise_type == 'asymmetric':
        print("asy: ", n, " ", noise_ratio)
        # CIFAR-10 specific label mapping
        mapping = {9: 1, 2: 0, 4: 7, 3: 5, 5: 3}
        for i in range(n):
            if random.random() < noise_ratio and labels[i].item() in mapping:
                # print("change")
                noisy_labels[i] = mapping[labels[i].item()]

    return noisy_labels
    
def verify_noise(original_labels, noisy_labels):
    assert len(original_labels) == len(noisy_labels), "Length of original and noisy labels must be equal"
    changed = (original_labels != noisy_labels).sum().item()
    total = len(original_labels)
    noise_ratio = changed / total
    return noise_ratio


# 
original_labels = torch.tensor([full_train_ds[i][1] for i in range(len(full_train_ds))])

# 
noise_ratio = 0.6  # 
noisy_labels = add_noise_to_labels(original_labels, noise_ratio, noise_type='symmetric')

# verify_noise
actual_noise_ratio = verify_noise(original_labels, noisy_labels)
print(f"Actual noise ratio: {actual_noise_ratio}")
print("Some original labels:", original_labels[:10])
print("Corresponding noisy labels (symmetric):", noisy_labels[:10])

asymmetric_mapping = {9: 1, 2: 0, 4: 7, 3: 5, 5: 3}

def calculate_asymmetric_noise_ratio(original_labels, noisy_labels, mapping):

    n = len(original_labels)
    actual_changes = 0
    potential_changes = 0

    for i in range(n):
        if original_labels[i].item() in mapping:
            potential_changes += 1
            if noisy_labels[i].item() == mapping[original_labels[i].item()]:
                actual_changes += 1

    if potential_changes == 0:
        return 0
    else:
        return actual_changes / potential_changes


noisy_labels_a = add_noise_to_labels(original_labels, noise_ratio, noise_type='asymmetric')
print("Some original labels:", original_labels[:10])
print("Corresponding noisy labels (asymmetric):", noisy_labels_a[:10])



actual_asymmetric_noise_ratio = calculate_asymmetric_noise_ratio(original_labels, noisy_labels_a, asymmetric_mapping)
print(f"Actual asymmetric noise ratio: {actual_asymmetric_noise_ratio}")



Actual noise ratio: 0.59838
Some original labels: tensor([6, 9, 9, 4, 1, 1, 2, 7, 8, 3])
Corresponding noisy labels (symmetric): tensor([9, 4, 7, 4, 6, 4, 4, 3, 1, 4])


In [37]:
def run_train_harness(noise_level=0.0, noise_type='symmetric'):
    # 重新加载数据集以避免标签被多次修改
    full_train_ds = torchvision.datasets.CIFAR10(root='../data', train=True, download=True, transform=transform_train)
    original_labels = torch.tensor(full_train_ds.targets)

    # 应用噪声
    if noise_level > 0:
        noisy_labels = add_noise_to_labels(original_labels, noise_level, noise_type)
        full_train_ds.targets = noisy_labels.tolist()

    # 分割训练集和验证集
    num_train = len(full_train_ds)
    num_valid = int(num_train * valid_ratio)
    train_ds, valid_ds = random_split(full_train_ds, [num_train - num_valid, num_valid])

    # 创建 DataLoader
    train_iter = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2)
    valid_iter = DataLoader(valid_ds, batch_size=batch_size, shuffle=False, num_workers=2)



    # 创建模型实例
    devices, num_epochs, lr, wd = d2l.try_all_gpus(), 18, 2e-4, 5e-4
    lr_period, lr_decay = 4, 0.9
    net1 = get_net().to(devices[0])
    net2 = get_net().to(devices[0])

    # 训练模型
    print("####### Training model with co-training: ########")
    train(net1, net2, train_iter, valid_iter, num_epochs, lr, wd, devices, lr_period, lr_decay)

# print(f"####### Training model with 0 noise level: ########")
# run_train_harness(0, 'symmetric')
# print("\n\n\n")

# noise_levels = [0.1, 0.3, 0.5, 0.8]
# noise_types = ['symmetric', 'asymmetric']

# for noise_type in noise_types:
#     for noise_level in noise_levels:
#         print(f"####### Training with {noise_type} noise level: {noise_level} ########")
#         run_train_harness(noise_level, noise_type)
#         print("\n\n\n")


In [23]:
# noise_types = ['symmetric', 'asymmetric']
# for noise_type in noise_types:
#     noise_level = 0.1
#     print(f"####### Training with {noise_type} noise level: {noise_level} ########")
#     run_train_harness(noise_level, noise_type)
#     print("\n\n\n")

In [24]:
noise_types = ['symmetric', 'asymmetric']
for noise_type in noise_types:
    noise_level = 0.3
    print(f"####### Training with {noise_type} noise level: {noise_level} ########")
    run_train_harness(noise_level, noise_type)
    print("\n\n\n")

####### Training with symmetric noise level: 0.3 ########
Files already downloaded and verified
####### Training model with co-training: ########
Epoch 1, Net1 loss 2.168, accuracy 0.216, Net2 loss 2.161, accuracy 0.216, Valid acc1 0.268, Valid acc2 0.271, time 96
Epoch 2, Net1 loss 2.037, accuracy 0.293, Net2 loss 2.036, accuracy 0.293, Valid acc1 0.305, Valid acc2 0.304, time 95
Epoch 3, Net1 loss 1.982, accuracy 0.327, Net2 loss 1.984, accuracy 0.327, Valid acc1 0.333, Valid acc2 0.327, time 95
Epoch 4, Net1 loss 1.942, accuracy 0.347, Net2 loss 1.945, accuracy 0.348, Valid acc1 0.346, Valid acc2 0.348, time 95
Epoch 5, Net1 loss 1.911, accuracy 0.368, Net2 loss 1.915, accuracy 0.367, Valid acc1 0.364, Valid acc2 0.355, time 95
Epoch 6, Net1 loss 1.888, accuracy 0.382, Net2 loss 1.889, accuracy 0.380, Valid acc1 0.364, Valid acc2 0.368, time 95
Epoch 7, Net1 loss 1.865, accuracy 0.395, Net2 loss 1.869, accuracy 0.391, Valid acc1 0.376, Valid acc2 0.376, time 95
Epoch 8, Net1 loss 1.

NameError: name 'metric' is not defined

In [38]:
noise_types = ['symmetric', 'asymmetric']
noise_types = [ 'asymmetric']
for noise_type in noise_types:
    noise_level = 0.3
    print(f"####### Training with {noise_type} noise level: {noise_level} ########")
    run_train_harness(noise_level, noise_type)
    print("\n\n\n")

####### Training with asymmetric noise level: 0.3 ########
Files already downloaded and verified
asy:  50000   0.3
####### Training model with co-training: ########
Epoch 1, Net1 loss 1.905, accuracy 0.307, Net2 loss 1.899, accuracy 0.308, Valid acc1 0.378, Valid acc2 0.388, time 96
Epoch 2, Net1 loss 1.569, accuracy 0.419, Net2 loss 1.572, accuracy 0.420, Valid acc1 0.435, Valid acc2 0.432, time 96
Epoch 3, Net1 loss 1.433, accuracy 0.468, Net2 loss 1.446, accuracy 0.464, Valid acc1 0.475, Valid acc2 0.474, time 96
Epoch 4, Net1 loss 1.336, accuracy 0.503, Net2 loss 1.354, accuracy 0.500, Valid acc1 0.492, Valid acc2 0.499, time 97
Epoch 5, Net1 loss 1.263, accuracy 0.530, Net2 loss 1.283, accuracy 0.521, Valid acc1 0.502, Valid acc2 0.516, time 96
Epoch 6, Net1 loss 1.208, accuracy 0.548, Net2 loss 1.221, accuracy 0.543, Valid acc1 0.540, Valid acc2 0.527, time 96
Epoch 7, Net1 loss 1.163, accuracy 0.564, Net2 loss 1.171, accuracy 0.563, Valid acc1 0.540, Valid acc2 0.544, time 96
Ep

In [39]:
noise_types = ['symmetric', 'asymmetric']
for noise_type in noise_types:
    noise_level = 0.5
    print(f"####### Training with {noise_type} noise level: {noise_level} ########")
    run_train_harness(noise_level, noise_type)
    print("\n\n\n")

####### Training with symmetric noise level: 0.5 ########
Files already downloaded and verified
####### Training model with co-training: ########
Epoch 1, Net1 loss 2.253, accuracy 0.165, Net2 loss 2.257, accuracy 0.165, Valid acc1 0.195, Valid acc2 0.201, time 100
Epoch 2, Net1 loss 2.199, accuracy 0.212, Net2 loss 2.203, accuracy 0.210, Valid acc1 0.220, Valid acc2 0.222, time 98
Epoch 3, Net1 loss 2.178, accuracy 0.231, Net2 loss 2.179, accuracy 0.231, Valid acc1 0.232, Valid acc2 0.231, time 99
Epoch 4, Net1 loss 2.160, accuracy 0.248, Net2 loss 2.162, accuracy 0.244, Valid acc1 0.245, Valid acc2 0.250, time 98
Epoch 5, Net1 loss 2.146, accuracy 0.256, Net2 loss 2.148, accuracy 0.255, Valid acc1 0.256, Valid acc2 0.252, time 98
Epoch 6, Net1 loss 2.136, accuracy 0.265, Net2 loss 2.137, accuracy 0.265, Valid acc1 0.269, Valid acc2 0.263, time 98
Epoch 7, Net1 loss 2.126, accuracy 0.273, Net2 loss 2.126, accuracy 0.270, Valid acc1 0.274, Valid acc2 0.268, time 97
Epoch 8, Net1 loss 2

In [40]:

noise_types = ['symmetric', 'asymmetric']
for noise_type in noise_types:
    noise_level = 0.8
    print(f"####### Training with {noise_type} noise level: {noise_level} ########")
    run_train_harness(noise_level, noise_type)
    print("\n\n\n")

####### Training with symmetric noise level: 0.8 ########
Files already downloaded and verified
####### Training model with co-training: ########
Epoch 1, Net1 loss 2.310, accuracy 0.106, Net2 loss 2.313, accuracy 0.104, Valid acc1 0.111, Valid acc2 0.107, time 95
Epoch 2, Net1 loss 2.303, accuracy 0.116, Net2 loss 2.305, accuracy 0.109, Valid acc1 0.109, Valid acc2 0.115, time 96
Epoch 3, Net1 loss 2.300, accuracy 0.119, Net2 loss 2.301, accuracy 0.117, Valid acc1 0.116, Valid acc2 0.115, time 96
Epoch 4, Net1 loss 2.298, accuracy 0.121, Net2 loss 2.298, accuracy 0.122, Valid acc1 0.117, Valid acc2 0.115, time 95
Epoch 5, Net1 loss 2.294, accuracy 0.128, Net2 loss 2.297, accuracy 0.125, Valid acc1 0.120, Valid acc2 0.115, time 95
Epoch 6, Net1 loss 2.294, accuracy 0.126, Net2 loss 2.293, accuracy 0.128, Valid acc1 0.116, Valid acc2 0.118, time 96
Epoch 7, Net1 loss 2.292, accuracy 0.130, Net2 loss 2.292, accuracy 0.131, Valid acc1 0.122, Valid acc2 0.122, time 97
Epoch 8, Net1 loss 2.

In [ ]:
noise_types = ['asymmetric']
for noise_type in noise_types:
    noise_level = 0.0
    print(f"####### Training with {noise_type} noise level: {noise_level} ########")
    run_train_harness(noise_level, noise_type)
    print("\n\n\n")

In [ ]:
noise_types = ['asymmetric', 'asymmetric']
for noise_type in noise_types:
    noise_level = 0.1
    print(f"####### Training with {noise_type} noise level: {noise_level} ########")
    run_train_harness(noise_level, noise_type)
    print("\n\n\n")